信用评分卡模型在Python中实践（上）
https://blog.csdn.net/kxiaozhuk/article/details/84612632

In [5]:
#####代码中需要引用的包#####
import numpy as np
import pandas as pd
from sklearn.utils import shuffle
from sklearn.feature_selection import RFE, f_regression
import scipy.stats.stats as stats
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
import math
import matplotlib.pyplot as plt


# 1.数据准备

In [14]:
import pandas as pd
import xlrd
import csv
import codecs
#xlsx转为csv
def xlsx_to_csv(path,doc_name):
    workbook = xlrd.open_workbook(path)
    table = workbook.sheet_by_index(0)
    with codecs.open(doc_name, 'w', encoding='utf-8') as f:
        write = csv.writer(f)
        for row_num in range(table.nrows):
            row_value = table.row_values(row_num)
            write.writerow(row_value)

if __name__ == '__main__':
    xlsx_to_csv('试题2数据.xlsx','data.csv')
#读取csv
import pandas as pd
data = pd.read_csv('data_f.csv',index_col=0,header=0)

In [15]:
data.dtypes

y                         float64
city                      float64
mar_status                float64
age                       float64
xaccount_age              float64
cred_limit                float64
this_bill_rate            float64
this_bill_mp              float64
last_mp_days              float64
six_bill_num              float64
six_bill_avg_amt          float64
six_cycle_mp_num          float64
six_cycle_mp_avg_amt      float64
six_bill_avg_debt_rate    float64
six_bill_low_repay_num    float64
epp_nbr_12m               float64
alop_nbr_12m              float64
epp_avg_amt_12m           float64
alop_avg_amt_12m          float64
consume_num_session12     float64
consume_num_session6      float64
consume_num_session3      float64
consume_num_session       float64
consume_amt_session12     float64
consume_amt_session6      float64
consume_amt_session3      float64
consume_amt_session       float64
month_avg_use_year        float64
month_avg_use_month6      float64
month_avg_use_

 接下来对数据集进行拆分，按照7:3拆分训练集和测试集，并将违约样本用“1”表示，正常样本用“0”表示。

In [16]:
#提取样本训练集和测试集
def split_data(data, ratio=0.7, seed=None):
    if seed:
        shuffle_data = shuffle(data, random_state=seed)
    else:
        shuffle_data = shuffle(data, random_state=np.random.randint(10000))
    train = shuffle_data.iloc[:int(ratio*len(shuffle_data)), ]
    test = shuffle_data.iloc[int(ratio*len(shuffle_data)):, ]
    return train, test
 
#设置seed是为了保证下次拆分的结果一致
df_train,df_test = split_data(data, ratio=0.7, seed=666)
#将违约样本用“1”表示，正常样本用“0”表示。
credit_risk = [0 if x=='0' else 1 for x in df_train['y']]
data = df_train
data['y']=credit_risk

AttributeError: 'list' object has no attribute 'info'

In [26]:
y=data['y'] # 提取出survived 列
#X=data.drop(['id','y','six_optial_mp_num','six_optial_mp_avg_amt'],axis=1) # 提取除去这三列的其它所有列
X=data.drop(['y'],axis=1) # 提取除去这三列的其它所有列
X.shape

from sklearn.cross_validation import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X,y, test_size=0.25, random_state=0) 

C:\Users\hxm\AppData\Roaming\Python\Python36\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [29]:
Y_train.shape

(7500,)

# 三、定量和定性指标筛选

In [30]:
#根据观测。指定定量(连续)和定性指标
X_train.columns

Index(['city', 'mar_status', 'age', 'xaccount_age', 'cred_limit',
       'this_bill_rate', 'this_bill_mp', 'last_mp_days', 'six_bill_num',
       'six_bill_avg_amt', 'six_cycle_mp_num', 'six_cycle_mp_avg_amt',
       'six_bill_avg_debt_rate', 'six_bill_low_repay_num', 'epp_nbr_12m',
       'alop_nbr_12m', 'epp_avg_amt_12m', 'alop_avg_amt_12m',
       'consume_num_session12', 'consume_num_session6', 'consume_num_session3',
       'consume_num_session', 'consume_amt_session12', 'consume_amt_session6',
       'consume_amt_session3', 'consume_amt_session', 'month_avg_use_year',
       'month_avg_use_month6', 'month_avg_use_month3'],
      dtype='object')

In [34]:
var_lianxu=['city', 'age', 'xaccount_age', 'cred_limit',
       'this_bill_rate', 'this_bill_mp', 'last_mp_days', 'six_bill_num',
       'six_bill_avg_amt', 'six_cycle_mp_num', 'six_cycle_mp_avg_amt',
       'six_bill_avg_debt_rate', 'six_bill_low_repay_num', 'epp_nbr_12m',
       'alop_nbr_12m', 'epp_avg_amt_12m', 'alop_avg_amt_12m',
       'consume_num_session12', 'consume_num_session6', 'consume_num_session3',
       'consume_num_session', 'consume_amt_session12', 'consume_amt_session6',
       'consume_amt_session3', 'consume_amt_session', 'month_avg_use_year',
       'month_avg_use_month6', 'month_avg_use_month3']

In [35]:
var_lisan=[ 'mar_status']

In [39]:
df_feature = pd.DataFrame(X_train,columns=var_lianxu)

In [40]:
df_feature.shape

(7500, 28)

In [41]:
df_feature = pd.DataFrame(data,columns=var_lianxu)
f_regression(df_feature,Y_train)
#输出逐步回归后得到的变量，选择P值<=0.1的变量
#quant_model_vars = ["duration","amount","age","installment_rate"]


ValueError: Found input variables with inconsistent numbers of samples: [10000, 7500]

In [21]:
quant_vars=df_train['mar_status','six_cycle_mp_num'].columns

KeyError: ('mar_status', 'six_cycle_mp_num')

In [11]:
quant_vars

Index(['id', 'y', 'city', 'mar_status', 'age', 'xaccount_age', 'cred_limit',
       'this_bill_rate', 'this_bill_mp', 'last_mp_days', 'six_bill_num',
       'six_bill_avg_amt', 'six_cycle_mp_num', 'six_cycle_mp_avg_amt',
       'six_bill_avg_debt_rate', 'six_bill_low_repay_num', 'six_optial_mp_num',
       'six_optial_mp_avg_amt', 'epp_nbr_12m', 'alop_nbr_12m',
       'epp_avg_amt_12m', 'alop_avg_amt_12m', 'consume_num_session12',
       'consume_num_session6', 'consume_num_session3', 'consume_num_session',
       'consume_amt_session12', 'consume_amt_session6', 'consume_amt_session3',
       'consume_amt_session', 'month_avg_use_year', 'month_avg_use_month6',
       'month_avg_use_month3'],
      dtype='object')